In [1]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
import torch
print(torch.__version__)
print(torch.cuda.is_available())

tcmalloc: large alloc 1073750016 bytes == 0x5ca88000 @  0x7f781968e2a4 0x594e17 0x626104 0x51190a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x4f3338 0x510fb0 0x5119bd 0x4f6070
0.4.0
True


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!ls "/content/drive/My Drive/Deepbind/"

 best_hyperpamarameters_2.pth
 best_hyperpamarameters.pth
'ELK1_GM12878_ELK1_(1277-1)_Stanford_AC.seq.gz'
'ELK1_GM12878_ELK1_(1277-1)_Stanford_B.seq.gz'
 GABPA_GM12878_GABP_HudsonAlpha_AC.seq.gz
 GABPA_GM12878_GABP_HudsonAlpha_B.seq.gz
 MyModel_2.pth
 MyModel.pth
 USF1_HepG2_USF-1_HudsonAlpha_AC.seq.gz
 USF1_HepG2_USF-1_HudsonAlpha_B.seq.gz
'ZBTB7A_HepG2_ZBTB7A_(SC-34508)_HudsonAlpha_AC.seq.gz'


In [4]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
       process = psutil.Process(os.getpid())
       print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
       print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Running setup.py bdist_wheel for gputil ... - done
  Stored in directory: /root/.cache/pip/wheels/17/0f/04/b79c006972335e35472c0b835ed52bfc0815258d409f560108
Successfully built gputil
  Running setup.py bdist_wheel for humanize ... - done
  Stored in directory: /root/.cache/pip/wheels/69/86/6c/f8b8593bc273ec4b0c653d3827f7482bb2001a2781a73b7f44
Successfully built humanize
Gen RAM Free: 12.9 GB  | Proc size: 209.0 MB
GPU RAM Free: 11430MB | Used: 11MB | Util   0% | Total 11441MB


In [0]:
import numpy as np
import csv
import math 
import random
import gzip
from scipy.stats import bernoulli
import torch
from sklearn import metrics

In [0]:
nummotif=16 #number of motifs to discover
bases='ACGT' #DNA bases
basesRNA='ACGU'#RNA bases
batch_size=64 #fixed batch size -> see notes to problem about it
dictReverse={'A':'T','C':'G','G':'C','T':'A','N':'N'} #dictionary to implement reverse-complement mode
reverse_mode=False

In [0]:
def seqtopad(sequence,motlen,kind='DNA'):
    rows=len(sequence)+2*motlen-2
    S=np.empty([rows,4])
    base= bases if kind=='DNA' else basesRNA
    for i in range(rows):
        for j in range(4):
            if i-motlen+1<len(sequence) and sequence[i-motlen+1]=='N' or i<motlen-1 or i>len(sequence)+motlen-2:
                S[i,j]=np.float32(0.25)
            elif sequence[i-motlen+1]==base[j]:
                S[i,j]=np.float32(1)
            else:
                S[i,j]=np.float32(0)
    return np.transpose(S)

In [0]:
def dinucshuffle(sequence):
    b=[sequence[i:i+2] for i in range(0, len(sequence), 2)]
    random.shuffle(b)
    d=''.join([str(x) for x in b])
    return d

In [0]:
def complement(seq):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'}
    complseq = [complement[base] for base in seq]
    return complseq
  
def reverse_complement(seq):
    seq = list(seq)
    seq.reverse()
    return ''.join(complement(seq))


In [0]:
class Chip():
    def __init__(self,filename,motiflen=24,reverse_complemet_mode=reverse_mode):
        self.file = filename
        self.motiflen = motiflen
        self.reverse_complemet_mode=reverse_complemet_mode
            
    def openFile(self):
        train_dataset=[]
        with gzip.open(self.file, 'rt') as data:
            next(data)
            reader = csv.reader(data,delimiter='\t')
            if not self.reverse_complemet_mode:
              for row in reader:
                      train_dataset.append([seqtopad(row[2],self.motiflen),[1]])
                      train_dataset.append([seqtopad(dinucshuffle(row[2]),self.motiflen),[0]])
            else:
              for row in reader:
                      train_dataset.append([seqtopad(row[2],self.motiflen),[1]])
                      train_dataset.append([seqtopad(reverse_complement(row[2]),self.motiflen),[1]])
                      train_dataset.append([seqtopad(dinucshuffle(row[2]),self.motiflen),[0]])
                      train_dataset.append([seqtopad(dinucshuffle(reverse_complement(row[2])),self.motiflen),[0]])
        #random.shuffle(train_dataset)
        train_dataset_pad=train_dataset

        size=int(len(train_dataset_pad)/3)
        firstvalid=train_dataset_pad[:size]
        secondvalid=train_dataset_pad[size:size+size]
        thirdvalid=train_dataset_pad[size+size:]
        firsttrain=secondvalid+thirdvalid
        secondtrain=firstvalid+thirdvalid
        thirdtrain=firstvalid+secondvalid
        return firsttrain,firstvalid,secondtrain,secondvalid,thirdtrain,thirdvalid,train_dataset_pad

In [0]:
# chipseq=Chip('/content/drive/My Drive/Deepbind/ZBTB7A_HepG2_ZBTB7A_(SC-34508)_HudsonAlpha_AC.seq.gz')
chipseq=Chip('/content/drive/My Drive/Deepbind/ELK1_GM12878_ELK1_(1277-1)_Stanford_AC.seq.gz')



In [0]:
train1,valid1,train2,valid2,train3,valid3,alldataset=chipseq.openFile()

In [0]:
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader

class chipseq_dataset(Dataset):
    """ Diabetes dataset."""

    def __init__(self,xy=None):
        self.x_data=np.asarray([el[0] for el in xy],dtype=np.float32)
        self.y_data =np.asarray([el[1] for el in xy ],dtype=np.float32)
        self.x_data = torch.from_numpy(self.x_data)
        self.y_data = torch.from_numpy(self.y_data)
        self.len=len(self.x_data)
      

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len



In [0]:
train1_dataset=chipseq_dataset(train1)
train2_dataset=chipseq_dataset(train2)
train3_dataset=chipseq_dataset(train3)
valid1_dataset=chipseq_dataset(valid1)
valid2_dataset=chipseq_dataset(valid2)
valid3_dataset=chipseq_dataset(valid3)
alldataset_dataset=chipseq_dataset(alldataset)

In [0]:
batchSize=64
if reverse_mode:
  train_loader1 = DataLoader(dataset=train1_dataset,batch_size=batchSize,shuffle=False)
  train_loader2 = DataLoader(dataset=train2_dataset,batch_size=batchSize,shuffle=False)
  train_loader3 = DataLoader(dataset=train3_dataset,batch_size=batchSize,shuffle=False)
  valid1_loader = DataLoader(dataset=valid1_dataset,batch_size=batchSize,shuffle=False)
  valid2_loader = DataLoader(dataset=valid2_dataset,batch_size=batchSize,shuffle=False)
  valid3_loader = DataLoader(dataset=valid3_dataset,batch_size=batchSize,shuffle=False)
  alldataset_loader=DataLoader(dataset=alldataset_dataset,batch_size=batchSize,shuffle=False)
else:
  train_loader1 = DataLoader(dataset=train1_dataset,batch_size=batchSize,shuffle=True)
  train_loader2 = DataLoader(dataset=train2_dataset,batch_size=batchSize,shuffle=True)
  train_loader3 = DataLoader(dataset=train3_dataset,batch_size=batchSize,shuffle=True)
  valid1_loader = DataLoader(dataset=valid1_dataset,batch_size=batchSize,shuffle=False)
  valid2_loader = DataLoader(dataset=valid2_dataset,batch_size=batchSize,shuffle=False)
  valid3_loader = DataLoader(dataset=valid3_dataset,batch_size=batchSize,shuffle=False)
  alldataset_loader=DataLoader(dataset=alldataset_dataset,batch_size=batchSize,shuffle=False)

train_dataloader=[train_loader1,train_loader2,train_loader3]
valid_dataloader=[valid1_loader,valid2_loader,valid3_loader]


In [0]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
import torch.nn.functional as F
# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001
def logsampler(a,b):
        x=np.random.uniform(low=0,high=1)
        y=10**((math.log10(b)-math.log10(a))*x + math.log10(a))
        return y
    
def sqrtsampler(a,b):
        
        x=np.random.uniform(low=0,high=1)
        y=(b-a)*math.sqrt(x)+a
        return y
      
# input of shape(batch_size,inp_chan,iW)
class ConvNet(nn.Module):
    def __init__(self, nummotif,motiflen,poolType,neuType,mode,dropprob,learning_rate,momentum_rate,sigmaConv,sigmaNeu,beta1,beta2,beta3,reverse_complemet_mode=reverse_mode):
      
        super(ConvNet, self).__init__()
        self.poolType=poolType
        self.neuType=neuType
        self.mode=mode
        self.reverse_complemet_mode=reverse_complemet_mode
        self.dropprob=dropprob
        self.learning_rate=learning_rate
        self.momentum_rate=momentum_rate
        self.sigmaConv=sigmaConv
        self.sigmaNeu=sigmaNeu
        self.beta1=beta1
        self.beta2=beta2
        self.beta3=beta3
        self.wConv=torch.randn(nummotif,4,motiflen).to(device)
        torch.nn.init.normal_(self.wConv,mean=0,std=self.sigmaConv)
        self.wConv.requires_grad=True
        
        
        self.wRect=torch.randn(nummotif).to(device)
        torch.nn.init.normal_(self.wRect)
        self.wRect=-self.wRect
        self.wRect.requires_grad=True
        
        
        if neuType=='nohidden':
            
            if poolType=='maxavg':
                self.wNeu=torch.randn(2*nummotif,1).to(device)
            else:
                self.wNeu=torch.randn(nummotif,1).to(device)
            self.wNeuBias=torch.randn(1).to(device)
            torch.nn.init.normal_(self.wNeu,mean=0,std=self.sigmaNeu)
            torch.nn.init.normal_(self.wNeuBias,mean=0,std=self.sigmaNeu)

        else:
            if poolType=='maxavg':
                self.wHidden=torch.randn(2*nummotif,32).to(device)
            else:
                
                self.wHidden=torch.randn(nummotif,32).to(device)
            self.wNeu=torch.randn(32,1).to(device)
            self.wNeuBias=torch.randn(1).to(device)
            self.wHiddenBias=torch.randn(32).to(device)
            torch.nn.init.normal_(self.wNeu,mean=0,std=self.sigmaNeu)
            torch.nn.init.normal_(self.wNeuBias,mean=0,std=self.sigmaNeu)
            torch.nn.init.normal_(self.wHidden,mean=0,std=0.3)
            torch.nn.init.normal_(self.wHiddenBias,mean=0,std=0.3)
            
  
            self.wHidden.requires_grad=True
            self.wHiddenBias.requires_grad=True
            #wHiddenBias=tf.Variable(tf.truncated_normal([32,1],mean=0,stddev=sigmaNeu)) #hidden bias for everything

        self.wNeu.requires_grad=True
        self.wNeuBias.requires_grad=True
        

        
        
    
   
    def divide_two_tensors(self,x):
        l=torch.unbind(x)

        list1=[l[2*i] for i in range(int(x.shape[0]/2))]
        list2=[l[2*i+1] for i in range(int(x.shape[0]/2))]
        x1=torch.stack(list1,0)
        x2=torch.stack(list2,0)
        return x1,x2
    def forward_pass(self,x,mask=None,use_mask=False):
        
        conv=F.conv1d(x, self.wConv, bias=self.wRect, stride=1, padding=0)
        rect=conv.clamp(min=0)
        maxPool, _ = torch.max(rect, dim=2)
        if self.poolType=='maxavg':
            avgPool= torch.mean(rect, dim=2)                          
            pool=torch.cat((maxPool, avgPool), 1)
        else:
            pool=maxPool
        if(self.neuType=='nohidden'):
            if self.mode=='training': 
                if  not use_mask:
                  mask=bernoulli.rvs(self.dropprob, size=len(pool[0]))
                  mask=torch.from_numpy(mask).float().to(device)
                pooldrop=pool*mask
                out=pooldrop @ self.wNeu
                out.add_(self.wNeuBias)
            else:
                out=self.dropprob*(pool @ self.wNeu)
                out.add_(self.wNeuBias)       
        else:
            hid=pool @ self.wHidden
            hid.add_(self.wHiddenBias)
            hid=hid.clamp(min=0)
            if self.mode=='training': 
                if  not use_mask:
                  mask=bernoulli.rvs(self.dropprob, size=len(hid[0]))
                  mask=torch.from_numpy(mask).float().to(device)
                hiddrop=hid*mask
                out=self.dropprob*(hid @ self.wNeu)
                out.add_(self.wNeuBias)
            else:
                out=self.dropprob*(hid @ self.wNeu)
                out.add_(self.wNeuBias) 
        return out,mask
       
    def forward(self, x):
        
        if not  self.reverse_complemet_mode:
            out,_=self.forward_pass(x)
            
        else:
            
            x1,x2=self.divide_two_tensors(x)
            out1,mask=self.forward_pass(x1)
            out2,_=self.forward_pass(x2,mask,True)
            out=torch.max(out1, out2)
     
        return out
    


In [20]:
best_AUC=0
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
# device='cpu'
learning_steps_list=[4000,8000,12000,16000,20000]
for number in range(5):
    
    pool_List=['max','maxavg']        
    random_pool=random.choice(pool_List)
    
    neuType_list=['hidden','nohidden']
    random_neuType=random.choice(neuType_list)
    dropoutList=[0.5,0.75,1.0] 
        
    dropprob=random.choice(dropoutList)
    
    learning_rate=logsampler(0.0005,0.05)
 
    momentum_rate=sqrtsampler(0.95,0.99)  

    sigmaConv=logsampler(10**-7,10**-3)   
    

    sigmaNeu=logsampler(10**-5,10**-2) 
    beta1=logsampler(10**-15,10**-3)
    beta2=logsampler(10**-10,10**-3)
    beta3=logsampler(10**-10,10**-3)
        
        

      
    model_auc=[[],[],[]]
    for kk in range(3):
        model = ConvNet(16,24,random_pool,random_neuType,'training',dropprob,learning_rate,momentum_rate,sigmaConv,sigmaNeu,beta1,beta2,beta3,reverse_complemet_mode=reverse_mode).to(device)
        if random_neuType=='nohidden':
            optimizer = torch.optim.SGD([model.wConv,model.wRect,model.wNeu,model.wNeuBias], lr=model.learning_rate,momentum=model.momentum_rate,nesterov=True)

        else:
            optimizer = torch.optim.SGD([model.wConv,model.wRect,model.wNeu,model.wNeuBias,model.wHidden,model.wHiddenBias], lr=model.learning_rate,momentum=model.momentum_rate,nesterov=True)

        train_loader=train_dataloader[kk]
        valid_loader=valid_dataloader[kk]
        learning_steps=0
        while learning_steps<=20000:
            model.mode='training'
            auc=[]
            for i, (data, target) in enumerate(train_loader):
                data = data.to(device)
                target = target.to(device)
                if model.reverse_complemet_mode:
                  target_2=torch.randn(int(target.shape[0]/2),1)
                  for i in range(target_2.shape[0]):
                    target_2[i]=target[2*i]
                  target=target_2.to(device)
                
     
                # Forward pass
                output = model(data)
                if model.neuType=='nohidden':
                    loss = F.binary_cross_entropy(torch.sigmoid(output),target)+model.beta1*model.wConv.norm()+model.beta3*model.wNeu.norm()

                else:
                    loss = F.binary_cross_entropy(torch.sigmoid(output),target)+model.beta1*model.wConv.norm()+model.beta2*model.wHidden.norm()+model.beta3*model.wNeu.norm()

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                learning_steps+=1


    
    
                if learning_steps% 4000==0:
       
                    with torch.no_grad():
                        model.mode='test'
                        auc=[]
                        for i, (data, target) in enumerate(valid_loader):
                            data = data.to(device)
                            target = target.to(device)
                            if model.reverse_complemet_mode:
                              target_2=torch.randn(int(target.shape[0]/2),1)
                              for i in range(target_2.shape[0]):
                                target_2[i]=target[2*i]
                              target=target_2.to(device)
                            # Forward pass
                            output = model(data)
                            pred_sig=torch.sigmoid(output)
                            pred=pred_sig.cpu().detach().numpy().reshape(output.shape[0])
                            labels=target.cpu().numpy().reshape(output.shape[0])
                            
                            auc.append(metrics.roc_auc_score(labels, pred))
#                         print(np.mean(auc))
                        model_auc[kk].append(np.mean(auc))
                        print('AUC performance when training fold number ',kk+1, 'using ',learning_steps_list[len(model_auc[kk])-1],'learning steps = ',np.mean(auc))
    
    print('                   ##########################################               ')
    for n in range(5):
        AUC=(model_auc[0][n]+model_auc[1][n]+model_auc[2][n])/3
        #print(AUC)
        if AUC>best_AUC:
            best_AUC=AUC
            best_learning_steps=learning_steps_list[n]
            best_LearningRate=model.learning_rate
            best_LearningMomentum=model.momentum_rate
            best_neuType=model.neuType
            best_poolType=model.poolType
            best_sigmaConv=model.sigmaConv
            best_dropprob=model.dropprob
            best_sigmaNeu=model.sigmaNeu
            best_beta1=model.beta1
            best_beta2=model.beta2
            best_beta3=model.beta3
            
print('best_poolType=',best_poolType)
print('best_neuType=',best_neuType)
print('best_AUC=',best_AUC)            
print('best_learning_steps=',best_learning_steps)      
print('best_LearningRate=',best_LearningRate)
print('best_LearningMomentum=',best_LearningMomentum)
print('best_sigmaConv=',best_sigmaConv)
print('best_dropprob=',best_dropprob)
print('best_sigmaNeu=',best_sigmaNeu)
print('best_beta1=',best_beta1)
print('best_beta2=',best_beta2)
print('best_beta3=',best_beta3)

best_hyperparameters = {'best_poolType': best_poolType,'best_neuType':best_neuType,'best_learning_steps':best_learning_steps,'best_LearningRate':best_LearningRate,
                        'best_LearningMomentum':best_LearningMomentum,'best_sigmaConv':best_sigmaConv,'best_dropprob':best_dropprob,
                        'best_sigmaNeu':best_sigmaNeu,'best_beta1':best_beta1, 'best_beta2':best_beta2,'best_beta3':best_beta3}
torch.save(best_hyperparameters, '/content/drive/My Drive/Deepbind/best_hyperpamarameters.pth')
            
 

cuda:0
AUC performance when training fold number  1 using  4000 learning steps =  0.7573384510206571
AUC performance when training fold number  1 using  8000 learning steps =  0.7527348832953806
AUC performance when training fold number  1 using  12000 learning steps =  0.7504746737590816
AUC performance when training fold number  1 using  16000 learning steps =  0.7475097656249999
AUC performance when training fold number  1 using  20000 learning steps =  0.742764288183149
AUC performance when training fold number  2 using  4000 learning steps =  0.7766792723548407
AUC performance when training fold number  2 using  8000 learning steps =  0.7593681422667891
AUC performance when training fold number  2 using  12000 learning steps =  0.7493275434713288
AUC performance when training fold number  2 using  16000 learning steps =  0.741010302152388
AUC performance when training fold number  2 using  20000 learning steps =  0.7338485901682318
AUC performance when training fold number  3 usin

In [0]:
# input of shape(batch_size,inp_chan,iW)
class ConvNet_test(nn.Module):
    def __init__(self,nummotif,motiflen,poolType,neuType,mode,learning_steps,learning_rate,learning_Momentum,sigmaConv,dropprob,sigmaNeu,beta1,beta2,beta3,reverse_complemet_mode):
        super(ConvNet_test, self).__init__()
        self.poolType=poolType
        self.neuType=neuType
        self.mode=mode
        self.learning_rate=learning_rate
        self.reverse_complemet_mode=reverse_complemet_mode
        self.momentum_rate=learning_Momentum
        self.sigmaConv=sigmaConv
        self.wConv=torch.randn(nummotif,4,motiflen).to(device)
        torch.nn.init.normal_(self.wConv,mean=0,std=self.sigmaConv)
        self.wConv.requires_grad=True
        self.wRect=torch.randn(nummotif).to(device)
        torch.nn.init.normal_(self.wRect)
        self.wRect=-self.wRect
        self.wRect.requires_grad=True
        self.dropprob=dropprob
        self.sigmaNeu=sigmaNeu
        self.wHidden=torch.randn(2*nummotif,32).to(device)
        self.wHiddenBias=torch.randn(32).to(device)
        if neuType=='nohidden':
            
            if poolType=='maxavg':
                self.wNeu=torch.randn(2*nummotif,1).to(device)
            else:
                self.wNeu=torch.randn(nummotif,1).to(device)
            self.wNeuBias=torch.randn(1).to(device)
            torch.nn.init.normal_(self.wNeu,mean=0,std=self.sigmaNeu)
            torch.nn.init.normal_(self.wNeuBias,mean=0,std=self.sigmaNeu)

        else:
            if poolType=='maxavg':
                self.wHidden=torch.randn(2*nummotif,32).to(device)
            else:
                
                self.wHidden=torch.randn(nummotif,32).to(device)
            self.wNeu=torch.randn(32,1).to(device)
            self.wNeuBias=torch.randn(1).to(device)
            self.wHiddenBias=torch.randn(32).to(device)
            torch.nn.init.normal_(self.wNeu,mean=0,std=self.sigmaNeu)
            torch.nn.init.normal_(self.wNeuBias,mean=0,std=self.sigmaNeu)
            torch.nn.init.normal_(self.wHidden,mean=0,std=0.3)
            torch.nn.init.normal_(self.wHiddenBias,mean=0,std=0.3)
            
  
            self.wHidden.requires_grad=True
            self.wHiddenBias.requires_grad=True
            #wHiddenBias=tf.Variable(tf.truncated_normal([32,1],mean=0,stddev=sigmaNeu)) #hidden bias for everything

        self.wNeu.requires_grad=True
        self.wNeuBias.requires_grad=True
        

        self.beta1=beta1
        self.beta2=beta2
        self.beta3=beta3
        

    
    def divide_two_tensors(self,x):
        l=torch.unbind(x)

        list1=[l[2*i] for i in range(int(x.shape[0]/2))]
        list2=[l[2*i+1] for i in range(int(x.shape[0]/2))]
        x1=torch.stack(list1,0)
        x2=torch.stack(list2,0)
        return x1,x2
    def forward_pass(self,x,mask=None,use_mask=False):
        
        conv=F.conv1d(x, self.wConv, bias=self.wRect, stride=1, padding=0)
        rect=conv.clamp(min=0)
        maxPool, _ = torch.max(rect, dim=2)
        if self.poolType=='maxavg':
            avgPool= torch.mean(rect, dim=2)                          
            pool=torch.cat((maxPool, avgPool), 1)
        else:
            pool=maxPool
        if(self.neuType=='nohidden'):
            if self.mode=='training': 
                if  not use_mask:
                    mask=bernoulli.rvs(self.dropprob, size=len(pool[0]))
                    mask=torch.from_numpy(mask).float().to(device)
                pooldrop=pool*mask
                out=pooldrop @ self.wNeu
                out.add_(self.wNeuBias)
            else:
                out=self.dropprob*(pool @ self.wNeu)
                out.add_(self.wNeuBias)       
        else:
            hid=pool @ self.wHidden
            hid.add_(self.wHiddenBias)
            hid=hid.clamp(min=0)
            if self.mode=='training': 
                if  not use_mask:
                    mask=bernoulli.rvs(self.dropprob, size=len(hid[0]))
                    mask=torch.from_numpy(mask).float().to(device)
                hiddrop=hid*mask
                out=self.dropprob*(hid @ self.wNeu)
                out.add_(self.wNeuBias)
            else:
                out=self.dropprob*(hid @ self.wNeu)
                out.add_(self.wNeuBias) 
        return out,mask
       
    def forward(self, x):
        
        if not  self.reverse_complemet_mode:
            out,_=self.forward_pass(x)
        else:
            
            x1,x2=self.divide_two_tensors(x)
            out1,mask=self.forward_pass(x1)
            out2,_=self.forward_pass(x2,mask,True)
            out=torch.max(out1, out2)
        
        return out
    


In [23]:

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
learning_steps_list=[4000,8000,12000,16000,20000]
best_AUC=0

best_hyperparameters = torch.load('/content/drive/My Drive/Deepbind/best_hyperpamarameters.pth')

best_poolType=best_hyperparameters['best_poolType']
best_neuType=best_hyperparameters['best_neuType']
best_learning_steps=best_hyperparameters['best_learning_steps']
best_LearningRate=best_hyperparameters['best_LearningRate']
best_dropprob=best_hyperparameters['best_dropprob']
best_LearningMomentum=best_hyperparameters['best_LearningMomentum']
best_sigmaConv=best_hyperparameters['best_sigmaConv']
best_sigmaNeu=best_hyperparameters['best_sigmaNeu']
best_beta1=best_hyperparameters['best_beta1']
best_beta2=best_hyperparameters['best_beta2']
best_beta3=best_hyperparameters['best_beta3']


for number_models in range(6):

  model = ConvNet_test(16,24,best_poolType,best_neuType,'training',best_learning_steps,best_LearningRate,best_LearningMomentum,best_sigmaConv,best_dropprob,best_sigmaNeu,best_beta1,best_beta2,best_beta3,reverse_complemet_mode=False).to(device)

  if model.neuType=='nohidden':
      optimizer = torch.optim.SGD([model.wConv,model.wRect,model.wNeu,model.wNeuBias], lr=model.learning_rate,momentum=model.momentum_rate,nesterov=True)

  else:
      optimizer = torch.optim.SGD([model.wConv,model.wRect,model.wNeu,model.wNeuBias,model.wHidden,model.wHiddenBias], lr=model.learning_rate,momentum=model.momentum_rate,nesterov=True)




  train_loader=alldataset_loader
  valid_loader=alldataset_loader
  learning_steps=0
  while learning_steps<=best_learning_steps:
  
      for i, (data, target) in enumerate(train_loader):
          data = data.to(device)
          target = target.to(device)
          if model.reverse_complemet_mode:
              target_2=torch.randn(int(target.shape[0]/2),1)
              for i in range(target_2.shape[0]):
                target_2[i]=target[2*i]
              target=target_2.to(device)
            # Forward pass
          output = model(data)
          
          if model.neuType=='nohidden':
              loss = F.binary_cross_entropy(torch.sigmoid(output),target)+model.beta1*model.wConv.norm()+model.beta3*model.wNeu.norm()

          else:
              loss = F.binary_cross_entropy(torch.sigmoid(output),target)+model.beta1*model.wConv.norm()+model.beta2*model.wHidden.norm()+model.beta3*model.wNeu.norm()
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          learning_steps+=1
          
  with torch.no_grad():
      model.mode='test'
      auc=[]
      for i, (data, target) in enumerate(valid_loader):
          data = data.to(device)
          target = target.to(device)
          if model.reverse_complemet_mode:
              target_2=torch.randn(int(target.shape[0]/2),1)
              for i in range(target_2.shape[0]):
                target_2[i]=target[2*i]
              target=target_2.to(device)
          # Forward pass
          output = model(data)
          pred_sig=torch.sigmoid(output)
          pred=pred_sig.cpu().detach().numpy().reshape(output.shape[0])
          labels=target.cpu().numpy().reshape(output.shape[0])

          auc.append(metrics.roc_auc_score(labels, pred))
  #             
      AUC_training=np.mean(auc)
      print('AUC for model ',number_models,' = ',AUC_training)
      if AUC_training>best_AUC:
          state = {'conv': model.wConv,'rect':model.wRect,'wHidden':model.wHidden,'wHiddenBias':model.wHiddenBias,'wNeu':model.wNeu,'wNeuBias':model.wNeuBias }
          torch.save(state, '/content/drive/My Drive/Deepbind/MyModel_2.pth')



            
 

cuda:0
AUC for model  0  =  0.8911117144381336
AUC for model  1  =  0.866348566254332
AUC for model  2  =  0.8811529328712616
AUC for model  3  =  0.889619608161661
AUC for model  4  =  0.8890761124614939
AUC for model  5  =  0.9180261579867796


In [24]:
checkpoint = torch.load('/content/drive/My Drive/Deepbind/MyModel_2.pth')
model = ConvNet_test(16,24,best_poolType,best_neuType,'test',best_learning_steps,best_LearningRate,best_LearningMomentum,best_sigmaConv,best_dropprob,best_sigmaNeu,best_beta1,best_beta2,best_beta3,reverse_complemet_mode=reverse_mode).to(device)
model.wConv=checkpoint['conv']
model.wRect=checkpoint['rect']
model.wHidden=checkpoint['wHidden']
model.wHiddenBias=checkpoint['wHiddenBias']
model.wNeu=checkpoint['wNeu']
model.wNeuBias=checkpoint['wNeuBias']

with torch.no_grad():
      model.mode='test'
      auc=[]
     
      for i, (data, target) in enumerate(valid_loader):
          data = data.to(device)
          target = target.to(device)
          if model.reverse_complemet_mode:
              target_2=torch.randn(int(target.shape[0]/2),1)
              for i in range(target_2.shape[0]):
                target_2[i]=target[2*i]
              target=target_2.to(device)
          # Forward pass
          output = model(data)
          pred_sig=torch.sigmoid(output)
          pred=pred_sig.cpu().detach().numpy().reshape(output.shape[0])
          labels=target.cpu().numpy().reshape(output.shape[0])

          auc.append(metrics.roc_auc_score(labels, pred))
  #                         
      AUC_training=np.mean(auc)
      print(AUC_training)



0.9180261579867796


In [0]:
class Chip_test():
    def __init__(self,filename,motiflen=24,reverse_complemet_mode=reverse_mode):
        self.file = filename
        self.motiflen = motiflen
        self.reverse_complemet_mode=reverse_complemet_mode
            
    def openFile(self):
        test_dataset=[]
        with gzip.open(self.file, 'rt') as data:
            next(data)
            reader = csv.reader(data,delimiter='\t')
            if not self.reverse_complemet_mode:
              for row in reader:
                      test_dataset.append([seqtopad(row[2],self.motiflen),[int(row[3])]])
            else:
              for row in reader:
                      test_dataset.append([seqtopad(row[2],self.motiflen),[int(row[3])]])
                      test_dataset.append([seqtopad(reverse_complement(row[2]),self.motiflen),[int(row[3])]])
            
        return test_dataset


In [0]:
chipseq_test=Chip_test('/content/drive/My Drive/Deepbind/ELK1_GM12878_ELK1_(1277-1)_Stanford_B.seq.gz')
test_data=chipseq_test.openFile()
test_dataset=chipseq_dataset(test_data)
batchSize=test_dataset.__len__()
test_loader = DataLoader(dataset=test_dataset,batch_size=batchSize,shuffle=False)

In [27]:

with torch.no_grad():
      model.mode='test'
      auc=[]
     
      for i, (data, target) in enumerate(test_loader):
          data = data.to(device)
          target = target.to(device)
          if model.reverse_complemet_mode:
              target_2=torch.randn(int(target.shape[0]/2),1)
              for i in range(target_2.shape[0]):
                target_2[i]=target[2*i]
              target=target_2.to(device)
          # Forward pass
          output = model(data)
          pred_sig=torch.sigmoid(output)
          pred=pred_sig.cpu().detach().numpy().reshape(output.shape[0])
          labels=target.cpu().numpy().reshape(output.shape[0])
          
          auc.append(metrics.roc_auc_score(labels, pred))
  #                         
      AUC_training=np.mean(auc)
      print('AUC on test data = ',AUC_training)

AUC on test data =  0.899808
